In [ ]:
#!/usr/bin/env python3.7
# -*- coding: utf-8 -*-
# coding: utf-8

In [ ]:
import tensorflow.compat.v1 as tf

In [ ]:
tf.disable_v2_behavior()
tf.disable_eager_execution()

In [ ]:
from IPython.utils.capture import capture_output
from IPython.display import Image, clear_output
from pprint import pprint, pformat
from os import path, makedirs
from tqdm.auto import tqdm
from random import randint
from typing import List
from enum import Enum
from os import path
import numpy as np
import json

In [ ]:
import gpt_2_simple as gpt2

In [ ]:
from util.keys import Runs, Actions, Models, Keys

In [ ]:
config = {
    "run_name":        Runs.umbrum,
    "action":          Actions.TRAIN,
    "model_name":      Models.M774,
}

In [ ]:
PROMPT = ""

In [ ]:
corpus = path.join('corpus', config["run_name"], f'{config["run_name"]}.{config["model_name"]}.npz').replace('+', '')

In [ ]:
if not(path.isfile(corpus)):
    gpt2.encode_dataset(
        file_path=path.join('corpus', config["run_name"], f'{config["run_name"]}.txt'),
        out_path=path.join('corpus', config["run_name"], f'{config["run_name"]}.{config["model_name"]}.npz'),
        model_name=config["model_name"],
    )

In [ ]:
config_train = {
    "dataset":                       corpus,
#    "steps":                         200,
    "model_name":                    config["model_name"],
    "batch_size":                    3,
    "learning_rate":                 0.000001,
    "accumulate_gradients":          20,
    "run_name":                      config["run_name"],
    "sample_every":                  20,
    "sample_length":                 620,
    "sample_num":                    1,
    "save_every":                    20,
    "print_every":                   1,
    "max_checkpoints":               3,
    "allow_huge":                    True,
    "only_train_transformer_layers": False,
    "use_memory_saving_gradients":   True,
}

In [ ]:
config_gen = {
    "run_name":          config["run_name"],
    "return_as_list":    True,
    "prefix":            Keys.START_KEY + PROMPT,
    "truncate":          Keys.END___KEY,
    "sample_delim":      ' SEPARATOR '.center(80, "-"),
    "nsamples":          12,
    "batch_size":        3,
    "length":            200,
    "temperature":       0.6,
    "include_prefix":    True,
}

In [ ]:
def clean(t: str) -> List[str]:
    return list(s for s in t.replace(Keys.END___KEY, '').rstrip().lstrip().strip().split(Keys.START_KEY) if s)

In [ ]:
def lclean(l: list):
    return [clean(t) for t in l]

In [ ]:
if not path.isdir(path.join("models", config["model_name"])):
    gpt2.download_gpt2(model_name=config["model_name"])

In [ ]:
if not(path.isfile(corpus)):
    gpt2.encode_dataset(
        file_path=path.join('corpus', config["run_name"], f'{config["run_name"]}.txt'),
        out_path=path.join('corpus', config["run_name"], f'{config["run_name"]}.{config["model_name"]}.npz'),
        model_name=config["model_name"],
    )

In [ ]:
sess = gpt2.start_tf_sess(server=None)

In [ ]:
if (config["action"] == Actions.PULL):
    for model in ["1558M", "774M", "355M", "345M", "124M", "117M"]:
        if not path.isdir(path.join("models", model)):
            gpt2.download_gpt2(model_name=model)

In [ ]:
if (config["action"] == Actions.GRAPH):
    dot = Digraph(comment='GPT-2')
    for op in tqdm(tf.get_default_graph().get_operations(), leave=False):
        op_name = op.name.replace('/', '_').replace(':', '_').lower()
        dot.node(op_name, op_name, shape='square')
        for i in op.inputs:
            i_name = i.name.replace('/', '_').replace(':', '_').lower()
            dot.node(i_name, i_name, shape="circle")
            dot.edge(i_name, op_name)
        for o in op.outputs:
            o_name = o.name.replace('/', '_').replace(':', '_').lower()
            dot.node(o_name, o_name, shape="circle")
            dot.edge(op_name, o_name)
    x = dot.render(filename='hiper_gpt2.dot')

In [ ]:
if   (config["action"] == Actions.TRAIN):
    gpt2.finetune(sess, **config_train)
elif (config["action"] == Actions.GENERATE):
    gpt2.load_gpt2(sess, run_name=config['run_name'])
    gpt2.generate(sess, **config_gen)